In [1]:
import time
import json
import os
from IPython.display import clear_output
from ai_core_sdk.ai_core_v2_client import AICoreV2Client
from ai_api_client_sdk.models.parameter_binding import ParameterBinding
from enum import Enum

# Inline credentials
with open('config.json') as f:
    credCF = json.load(f)

# Set environment variables
def set_environment_vars(credCF):
    env_vars = {
        'AICORE_AUTH_URL': credCF['url'] + '/oauth/token',
        'AICORE_CLIENT_ID': credCF['clientid'],
        'AICORE_CLIENT_SECRET': credCF['clientsecret'],
        'AICORE_BASE_URL': credCF["serviceurls"]["AI_API_URL"] + "/v2",
        'AICORE_RESOURCE_GROUP': "default" 
    }    

    for key, value in env_vars.items():
            os.environ[key] = value    

# Create AI Core client instance
def create_ai_core_client(credCF):
    set_environment_vars(credCF)  # Ensure environment variables are set
    return AICoreV2Client(
        base_url=os.environ['AICORE_BASE_URL'],
        auth_url=os.environ['AICORE_AUTH_URL'],
        client_id=os.environ['AICORE_CLIENT_ID'],
        client_secret=os.environ['AICORE_CLIENT_SECRET'],
        resource_group=os.environ['AICORE_RESOURCE_GROUP']
    )

ai_core_client = create_ai_core_client(credCF)

In [2]:
AI_API_URL = "https://api.ai.prod.eu-central-1.aws.ml.hana.ondemand.com/v2/inference/deployments/dcecde8d10aeb6a2"

In [3]:
from gen_ai_hub.proxy.langchain.openai import ChatOpenAI
from gen_ai_hub.proxy.core.proxy_clients import get_proxy_client
from gen_ai_hub.proxy.langchain.init_models import init_embedding_model

proxy_client = get_proxy_client('gen-ai-hub')

chat_llm = ChatOpenAI(proxy_model_name='gpt-4o', proxy_client=proxy_client)

Anthropic FTW!

In [4]:
import os
import uuid
import tempfile
import shutil
import gradio as gr
from typing import Dict, List, Tuple
import chromadb
from chromadb.utils import embedding_functions
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings
from langchain.document_loaders import (
    TextLoader )
from langchain.chat_models import ChatOpenAI
from langchain.chains import ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory

In [5]:
# Dictionary to store session data
sessions = {}

In [6]:
# Pre-defined prompts
PREDEFINED_PROMPTS = {
    "Summarize Document": "Please provide a concise summary of the main points in these documents.",
    "Generate a Report": """Generate a summary of the contents provided earlier with the following in a markdown format:
     It should includes all details from all discussions, organized by topic. The resulting detailed meeting notes should include all details for all topics discussed so that someone who was not present at the meeting can understand both the context of the topic and have all supporting details organized and grouped in logically related topics without having read the full transcript. Deliver the detailed notes using a professional and neutral tone and in an easy-to-read format by using headlines, subheads, and bullet points.  Include any meeting participant introductions at the top of the meeting notes and include all details from the introduction that are provided including name, role or title, history, background, location, and any other specific information provided. Also include a list of meeting participants and their roles if these can be determined in the meeting transcript at the top of the meeting notes, grouped by the participants stated or determined organization. At the end of the meeting notes, include a comprehensive list of parking lot actions. Parking lot actions, or parking lots, or open topics . Parling lot or open topics are topics that cannot be covered in the current meeting and need to be captured for later action. Parking lots include any activities that are either explicitly stated or implicitly stated or suggested, as an action to be taken, follow-up activity, or next step. Look for the following common phrases or terms that can be used to identify the list of action items: "parking lot", "action item", "follow-up", "let me take that", "let me get back with you", "check on that", "Open Topic", "Open Point", "need to check", "not sure", "send that", "let me confirm", "table that", etc. For each parking lot item, include the target completion date and who is responsible . If this information is included or can be determined in the meeting transcript. In another outline Issues and the corresponding action , or a something that was highlighted as challenge to which a recommendation or solution was provided or discussed, capture this also under Issues and Actions section. Capture the customer process pain points in another section. Pain points are challenges within the current processes that are typically manually intensive or inadequate, something not working right , something stated as a challenge etc.  In another section, include the high-level requirements that include any stated capability, functionality or process that is required for the new solution. In another section create a Summary as bullet points on what was discussed in the meeting.  Review the transcript from the beginning to the end and then re-read the transcript from the end back to the start to ensure that no details are missed. The meeting notes should only include information determined from the earlier provided documents, so do not add or make up any information that may be missing. Use the instructions for the content in the attached document."""
    # "Find Key Facts": "What are the key facts and figures mentioned in these documents?",
    # "Explain Concept": "Explain the concept of {concept} as discussed in these documents.",
    # "Compare Ideas": "Compare and contrast the different perspectives on {topic} presented in these documents.",
    # "Extract Action Items": "What are the action items or next steps mentioned in these documents?",
    # "Identify Problems": "What problems or challenges are identified in these documents?",
    # "Provide Recommendations": "Based on these documents, what recommendations can you provide about {subject}?"
}

In [7]:
def get_loader_for_file(file_path):
    """Return the appropriate loader based on file extension"""
    ext = os.path.splitext(file_path)[1].lower()
    if ext == '.txt':
        return TextLoader(file_path)
    else:
        raise ValueError(f"Unsupported file type: {ext}")

In [8]:
def process_files(session_id, files):
    """Process uploaded files and create/update the vector store"""
    # Create temp directory for this session if it doesn't exist
    if not os.path.exists(f"temp_{session_id}"):
        os.makedirs(f"temp_{session_id}")
    
    # Save uploaded files to temp directory
    file_paths = []
    for file in files:
        temp_path = os.path.join(f"temp_{session_id}", os.path.basename(file.name))
        shutil.copy(file.name, temp_path)
        file_paths.append(temp_path)
    
    # Process all documents
    documents = []
    for file_path in file_paths:
        try:
            loader = get_loader_for_file(file_path)
            documents.extend(loader.load())
        except Exception as e:
            print(f"Error loading {file_path}: {str(e)}")
    
    # Split documents
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=1000,
        chunk_overlap=100
    )
    split_documents = text_splitter.split_documents(documents)
    
    # Create or update vector store
    embedding = init_embedding_model('text-embedding-ada-002')
    
    # If vector store already exists for this session, add to it
    persist_directory = f"chroma_db_{session_id}"
    if os.path.exists(persist_directory):
        vector_store = Chroma(
            persist_directory=persist_directory,
            embedding_function=embedding
        )
        vector_store.add_documents(split_documents)
    else:
        # Create new vector store
        vector_store = Chroma.from_documents(
            documents=split_documents,
            embedding=embedding,
            persist_directory=persist_directory
        )
    
    vector_store.persist()
    
    # Create or update retrieval chain
    memory = ConversationBufferMemory(
        memory_key="chat_history",
        return_messages=True
    )
    
    # llm = ChatOpenAI(temperature=0, model_name="gpt-3.5-turbo")
    retrieval_chain = ConversationalRetrievalChain.from_llm(
        llm=chat_llm,
        retriever=vector_store.as_retriever(),
        memory=memory
    )
    
    # Update session data
    if session_id not in sessions:
        sessions[session_id] = {
            "vector_store": vector_store,
            "retrieval_chain": retrieval_chain,
            "file_paths": file_paths,
            "chat_history": []
        }
    else:
        sessions[session_id]["vector_store"] = vector_store
        sessions[session_id]["retrieval_chain"] = retrieval_chain
        sessions[session_id]["file_paths"].extend(file_paths)
    
    return f"Successfully processed {len(files)} files."

In [9]:
def reset_session(session_id):
    """Delete all files and reset the session"""
    if session_id in sessions:
        # Clean up temp directory
        if os.path.exists(f"temp_{session_id}"):
            shutil.rmtree(f"temp_{session_id}")
        
        # Clean up vector store
        if os.path.exists(f"chroma_db_{session_id}"):
            shutil.rmtree(f"chroma_db_{session_id}")
        
        # Remove session data
        del sessions[session_id]
    
    return "Session reset successfully. You can upload new files now."

In [10]:
def chat(session_id, query, history):
    """Process a query using the retrieval chain and update chat history"""
    if session_id not in sessions:
        return "Please upload some files first.", history
    
    try:
        # Get response from retrieval chain
        response = sessions[session_id]["retrieval_chain"].run(query)
        
        # Update chat history
        sessions[session_id]["chat_history"].append((query, response))
        
        # Update the Gradio chat history
        history.append((query, response))
        
        return "", history
    except Exception as e:
        return f"Error: {str(e)}", history

In [11]:
def use_predefined_prompt(prompt_key: str, concept_input: str, history: List[Tuple[str, str]], session_id: str):
    """Handle a predefined prompt selection"""
    if not prompt_key or prompt_key not in PREDEFINED_PROMPTS:
        return history, ""
    
    # Get the template and format it if needed
    template = PREDEFINED_PROMPTS[prompt_key]
    if "{concept}" in template and concept_input:
        prompt = template.replace("{concept}", concept_input)
    elif "{topic}" in template and concept_input:
        prompt = template.replace("{topic}", concept_input)
    elif "{subject}" in template and concept_input:
        prompt = template.replace("{subject}", concept_input)
    else:
        prompt = template
        
    # Return the prompt to be inserted in the textbox
    return history, prompt

In [12]:
def create_chat_interface():
    """Create a Gradio interface for the chatbot"""
    with gr.Blocks() as demo:
        session_id = gr.State(lambda: str(uuid.uuid4()))
        
        gr.Markdown("# ISD Hub Service Report Accelerator")#("# Personalized RAG Chatbot")
        gr.Markdown("Upload files and generate a well formatted report or simply just ask questions about their content.")
        
        with gr.Row():
            with gr.Column(scale=2):
                file_upload = gr.File(file_count="multiple", label="Upload Text Files")
                upload_button = gr.Button("Process Files")
                reset_button = gr.Button("Reset Session (Delete All Files)")
                status = gr.Textbox(label="Status", interactive=False)
            
        chatbot = gr.Chatbot(label="Chat History")
        gr.Markdown("## Predefined Prompts")
        with gr.Row():
            prompt_dropdown = gr.Dropdown(
                choices=list(PREDEFINED_PROMPTS.keys()),
                label="Select a prompt template"
            )
            concept_input = gr.Textbox(
                label="Concept/Topic/Subject (if needed)",
                placeholder="Enter a concept, topic or subject"
            )
        use_prompt_button = gr.Button("Use Selected Prompt")

        msg = gr.Textbox(label="Ask a question about your documents")
        # show a clear button, which clears the chat history and starts a new chat session
        clear_button = gr.Button("Clear Chat History")

        # Implement a click event for the clear button
        clear_button.click(
            fn=lambda: ([]),  # Clear the chat history
            inputs=[],
            outputs=[chatbot]
        )
        
        upload_button.click(
            fn=process_files, 
            inputs=[session_id, file_upload], 
            outputs=status
        )
        
        reset_button.click(
            fn=reset_session,
            inputs=[session_id],
            outputs=status
        )

        use_prompt_button.click(
        use_predefined_prompt,
        inputs=[prompt_dropdown, concept_input, chatbot, session_id],
        outputs=[chatbot, msg]
        )
        
        msg.submit(
            fn=chat,
            inputs=[session_id, msg, chatbot],
            outputs=[msg, chatbot]
        )
        
    return demo

In [13]:
demo = create_chat_interface()
demo.launch(share=True)

* Running on local URL:  http://127.0.0.1:7860

Could not create share link. Missing file: /Users/I583877/.cache/huggingface/gradio/frpc/frpc_darwin_arm64_v0.3. 

Please check your internet connection. This can happen if your antivirus software blocks the download of this file. You can install manually by following these steps: 

1. Download this file: https://cdn-media.huggingface.co/frpc-gradio-0.3/frpc_darwin_arm64
2. Rename the downloaded file to: frpc_darwin_arm64_v0.3
3. Move the file to this location: /Users/I583877/.cache/huggingface/gradio/frpc
